# Reporte

## Paralelización de Hill Climbing implementado con Numba


Numba nos permite, gracias a JIT, hacer compilación durante la ejecución. Entonces menos tiempo dedicado a la compilación inicial significa que el código se puede interpretar mucho más rápido.  y la interpretación automática de los tipos de datos lo que permite una interpretación más rápida y eficaz del código.

Dada la naturaleza de Hill Climbing consideramos que al implementar una paralelización resultaría en un aceleramiento del proceso iterativo inherente al algoritmo. Ya que nuestro algoritmo busca encontrar la ruta más adecuada, y con ayuda de la paralelización ofrecida por numba nos ayudaría a correr simultáneamente soluciones propuestas al algoritmo.

In [2]:
import os
import time
import numpy as np
import pandas as pd
os.chdir("../")
from src.hill_cg.hill import best_solution

In [4]:
dat = pd.read_csv("datasets/ca4663.tsp", names = ["index","uno","dos"], sep = " ")
dat.drop(['index'],axis = 1, inplace = True)
dat.dropna(inplace = True)
dat1 = dat.to_numpy()
dat2 = dat1[0:17,:]

En prmer lugar correremos la función `best_solution` sin Numba, , la cual nos retorna la ruta más corta encontrada por el algoritmo:

In [5]:
%timeit -n 3 -r 7 best_solution(dat2, 0, 1e-7, 100)

888 ms ± 19.2 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [21]:
distancia, ruta, tiempo_ejec = best_solution(dat2, 0, 1e-7, 100)

Aquí se muestra los resultados si Numba:

In [11]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3337.113634493085
[0, 1, 5, 6, 12, 13, 11, 15, 9, 16, 14, 7, 2, 4, 10, 8, 3, 0]
0.9038689136505127


In [20]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3353.4536058912754
[0, 3, 2, 4, 9, 16, 14, 7, 10, 8, 15, 11, 13, 12, 6, 5, 1, 0]
5.282973051071167


In [22]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3266.456452336076
[0, 1, 5, 6, 12, 13, 11, 15, 8, 10, 9, 16, 14, 7, 2, 4, 3, 0]
0.8811595439910889


Ahora corremos la misma función implementada con Numba:

In [6]:
from notebooks.nb_hill import nb_best_solution

In [7]:
%timeit -n 3 -r 7 nb_best_solution(dat2, 0, 1e-7, 100)

/home/ubuntu/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2119: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sol' of function 'calculate_distance'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "notebooks/nb_hill.py", line 51:
@nb.njit(parallel=True)
def calculate_distance(matrix, sol):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/ubuntu/.local/lib/python3.8/site-packages/numba/core/ir_utils.py:2119: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'sol' of function '__numba_parfor_gufunc_0x7fbf954be2b0'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<strin

5.3 s ± 127 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


Volvemos a correr el `best_solution` con Numba ya compilado el código.

In [8]:
%timeit -n 3 -r 7 nb_best_solution(dat2, 0, 1e-7, 100)

5.33 s ± 83.6 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


Aquí se muestran los resultados con Numba:

In [19]:
distancia, ruta, tiempo_ejec = nb_best_solution(dat2, 0, 1e-7, 100)

In [14]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3332.348304772912
[0, 1, 5, 6, 12, 13, 11, 15, 10, 8, 9, 14, 16, 7, 2, 4, 3, 0]
5.391537666320801


In [16]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3341.316828737772
[0, 3, 4, 2, 7, 9, 14, 16, 10, 8, 15, 11, 13, 12, 6, 5, 1, 0]
5.332757234573364


In [18]:
print(distancia)
print(ruta)
print(tiempo_ejec)

3269.2653147263823
[0, 1, 5, 6, 12, 13, 11, 15, 8, 10, 9, 14, 16, 7, 2, 4, 3, 0]
5.427420616149902


## Conclusión

Notamos que los tiempos son más pequeños en el primer, en el segundo ejemplo aumentaron los tiempos de ejecución en más de 5x.  La segunda implementación está basada en los datos mostrados del perfilamiento, donde se muestra que la función que más es llamada es `calculate_distance`, por ello tomamos la decisión de paralizar dicha función. 

A pesar de haber tomando dicha acción, ni los tiempos de ejecución y la solución dada por el algoritmo, cómo la ruta más corta, mostraron mejoras sustanciales con esta implementación. 

Dado estos resultados se tomó la decisión de modificar nuestro código para la práctica final, con el objetivo de utilizar frameworks cómo `dask` y el modulo de python, `multiprocessing`, par implementar el computo en paralelo que nos ayudara a tener mejores resultados. Además para acelerar la ejecución del algorimtmo realizaremos la implementación de la paralelización en GPU en un contenedor con Kale.